# CONVOLUTION

# 1a) Convolution
Four different implementations of circular convolution are introduced.

### Simple Linear Convolution
For simplicity, we first define simple linear convolution functions.  
These functions will be used in the final four implementations.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# circular convolution both temporal and circular

def circ_conv(x, h, domain):
    def circ_conv(x, h):
        r = np.zeros(N)
        for i in range(N):
            for j in range(i, i + N):
                r[j % N] += h[i] * x[j - i]
        return r
    
    def circ_CONV(x, h):
        return idft(dft(x) * dft(h))

    N = max(len(x), len(h))
    x_ = np.concatenate((x, np.array([0]*(N - len(x)))))
    h_ = np.concatenate((h, np.array([0]*(N - len(h)))))
    if domain == 'temporal':
        return circ_conv(x_, h_)
    elif domain == 'spectral':
        return circ_CONV(x_, h_)

In [ ]:
def circ_conv(x,h):
    #circular convolution in time domain
    #period 
    N = max(len(x),len(h))
    #append 0's if signals are of different length
    if len(x) > len(h):
        for i in range(0,(len(x)-len(h))):
            h = h.append(0)
    elif len(x) > len(h):
        x = x.append(0)*(len(h)-len(x))
    y = []
    #apply formular given in extra pdf
    for n in range(0,N):
        sum_ = 0
        for m in range(0,N-1):
            sum_ += x[m]* h[(n-m)%N]
        y.append(sum_)
    return y

In [3]:
def convolve(x, h, start, length):
    # create initial y output
    y = [0] * length;

    # calculate output
    for yKey in range(start, (start + length)):
        yOut = 0;

        # multiply x*h and add to product
        for xKey in range(len(x)):
            hKey = yKey - xKey;
            if hKey >= 0 and hKey < len(h):
                #print(f"{yKey}: {xKey} - {hKey}");
                yOut += x[xKey] * h[hKey];

        # set y
        y[yKey - start] = yOut;

    # return y
    return y;

def convolve_linear(x, h, overlap=False):

    # calculate the final maximum length
    if overlap:
        start = min(len(x),len(h)) - 1;
        length = max(len(x),len(h)) - min(len(x),len(h)) + 1;

    else:
        start = 0;
        length = len(x) + len(h) - 1;

    #print(f"length: {length}, start: {start}");

    return convolve(x, h, start, length);

### Circular Convolution By Multiplying Fourier Transform

In [4]:
def convolve_fft(x, h, overlap=False, periodic=False):

    if periodic:
        # Make x & h the same length
        if len(x) > len(h):
            h = np.append(h, [0] * (len(x)-len(h)));
        elif len(h) > len(x):
            x = np.append(x, [0] * (len(h)-len(x)));

        y = np.fft.ifft( np.fft.fft(x)*np.fft.fft(h) );
        y = np.real(y);
        y = np.round(y, 2);
        return y;

    else:
        return convolve_linear(x, h, overlap);

### Circular Convolution By Matrix Multiplication

In [5]:
def convolve_matrix(x, h, overlap=False, periodic=False):

    if periodic:
        # Make x & h the same length
        if len(x) > len(h):
            h = np.append(h, [0] * (len(x)-len(h)));
        elif len(h) > len(x):
            x = np.append(x, [0] * (len(h)-len(x)));

        N = len(x);
        buffer = np.empty((N, N*2));
        buffer[:,:N] = x;
        buffer[:,N:] = x[:N];

        matrix = np.ndarray((N,N));
        for i in range(N):
            matrix[i] = buffer[i][N-i:N*2-i];

        #print(matrix);

        y = np.dot(h, matrix);
        return y;

    else:
        return convolve_linear(x, h, overlap);

### Circular Convolution By Extension

In [6]:
def convolve_extend(x, h, overlap=False, periodic=False):

    if periodic:
        # Make x & h the same length
        if len(x) > len(h):
            h = np.append(h, [0] * (len(x)-len(h)));
        elif len(h) > len(x):
            x = np.append(x, [0] * (len(h)-len(x)));

        h = np.concatenate((h,h));
        start = len(x);
        length = len(x);

        return convolve(x, h, start, length);

    else:
        return convolve_linear(x, h, overlap);

### Circular Convolution By Addition (Modulo)

In [7]:
def convolve_add(x, h, overlap=False, periodic=False):

    if periodic:
        # Make x & h the same length
        if len(x) > len(h):
            h = np.append(h, [0] * (len(x)-len(h)));
        elif len(h) > len(x):
            x = np.append(x, [0] * (len(h)-len(x)));

        yFull = convolve_linear(x, h, False);

        y = [0] * len(x);
        for k in range(len(yFull)):
            y[k % len(x)] += yFull[k];

        return y;

    else:
        return convolve_linear(x, h, overlap);

### Check Output Of Different Convolution Functions

In [8]:
x = [1,2,3,4,5,6,7,8,9,10];
h = [1,0,2];

print("Linear Convolution");
print(np.convolve(x,h,mode="full"))
print(convolve_fft(x,h,False,False))
print(convolve_matrix(x,h,False,False))
print(convolve_extend(x,h,False,False))
print(convolve_add(x,h,False,False))

print("Linear Convolution (Overlap)");
print(np.convolve(x,h,mode="valid"))
print(convolve_fft(x,h,True,False))
print(convolve_matrix(x,h,True,False))
print(convolve_extend(x,h,True,False))
print(convolve_add(x,h,True,False))

print("Circular Convolution")
print(convolve_fft(x,h,True,True))
print(convolve_matrix(x,h,True,True))
print(convolve_extend(h,x,True,True))
print(convolve_add(x,h,True,True))

Linear Convolution
[ 1  2  5  8 11 14 17 20 23 26 18 20]
[1, 2, 5, 8, 11, 14, 17, 20, 23, 26, 18, 20]
[1, 2, 5, 8, 11, 14, 17, 20, 23, 26, 18, 20]
[1, 2, 5, 8, 11, 14, 17, 20, 23, 26, 18, 20]
[1, 2, 5, 8, 11, 14, 17, 20, 23, 26, 18, 20]
Linear Convolution (Overlap)
[ 5  8 11 14 17 20 23 26]
[5, 8, 11, 14, 17, 20, 23, 26]
[5, 8, 11, 14, 17, 20, 23, 26]
[5, 8, 11, 14, 17, 20, 23, 26]
[5, 8, 11, 14, 17, 20, 23, 26]
Circular Convolution
[19. 22.  5.  8. 11. 14. 17. 20. 23. 26.]
[19. 22.  5.  8. 11. 14. 17. 20. 23. 26.]
[19, 22, 5, 8, 11, 14, 17, 20, 23, 26]
[19, 22, 5, 8, 11, 14, 17, 20, 23, 26]


In [ ]:
# phase shift on ideal low pass filter

def apply_linear_phase(M, H, phase):
    H_new = np.zeros(M, dtype='complex')
    H_new[:M//2] = H[:M//2] * np.exp(-1j * phase/(2*np.pi) * np.arange(M//2))
    H_new[M//2+1:] = H[M//2+1:] * np.exp(-1j * phase/(2*np.pi) * np.arange(-M//2, -1))
    return H_new

In [3]:
def square(func):
    def wrapper(a,b):
        return func(a,b) *func(a,b)
    return wrapper

@square
def add(a,b):
    return a+b

print(add(5,3))

64
